FYP 2021 S2 
Jingji Du
Walking coach 2D simulation and RL network

In [1]:
#set up of the package 
# the package need 
# Ray, Tensorflow, gym, numpy


%matplotlib

import numpy as np 
import random
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation 
import matplotlib.animation as animation 

import math

import logging
from typing import List, Optional, Type

import logging
from typing import List, Optional, Type

import ray 

from ray.rllib.agents import ppo,ars

from ray.rllib.agents.dqn.dqn_tf_policy import DQNTFPolicy
from ray.rllib.agents.dqn.dqn_torch_policy import DQNTorchPolicy
from ray.rllib.agents.dqn.simple_q import SimpleQTrainer, \
    DEFAULT_CONFIG as SIMPLEQ_DEFAULT_CONFIG
from ray.rllib.agents.trainer import Trainer
from ray.rllib.evaluation.worker_set import WorkerSet
from ray.rllib.execution.concurrency_ops import Concurrently
from ray.rllib.execution.metric_ops import StandardMetricsReporting
from ray.rllib.execution.replay_buffer import LocalReplayBuffer
from ray.rllib.execution.replay_ops import Replay, StoreToReplayBuffer
from ray.rllib.execution.rollout_ops import ParallelRollouts
from ray.rllib.execution.train_ops import TrainOneStep, UpdateTargetNetwork, \
    MultiGPUTrainOneStep
from ray.rllib.policy.policy import LEARNER_STATS_KEY, Policy
from ray.rllib.utils.typing import TrainerConfigDict
from ray.util.iter import LocalIterator

import gym
from gym import Env
from gym.spaces import Discrete, Box

import tensorflow
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,Conv3D
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

import ray
import os
from ray import tune
from ray.rllib import agents
from ray.tune.registry import register_env
from itertools import count
import pprint



global inital_energy
global inital_speed


global Goal
global person_speed_rate
global person_moving_rate
global Action_space 

# global variable 
person_speed_rate = {"a" : 0.8, "b" :0.7 , "c" : 0.95, "d" : 0.55, "e" : 0.65}
person_moving_rate = {"a" : 0.5, "b" :0.6 , "c" : 0.3, "d" : 0.4, "e" : 0.7}
Action_space = ["A","B","C","D","E"]
Goal = 500

Reward_epic = []
Energy_epic = []
Speed_epic = []

global Reward_epic
global Energy_epic
global Speed_epic




Using matplotlib backend: Qt5Agg


Here are the Dictionary used for Personal type and Orientation.

The Person type defines The impact of the action. 

Example:
If the robot pickup action A for Person A, the energy of person A will increase 40 and orientaton will increse 1


In [2]:
Person_a = {"A" : 40, "B": -25, "C" : -10, "D":15, "E" :-5}
Person_b = {"A" : 35, "B": -20, "C" :-15, "D":22, "E" :-3}
Person_c = {"A" : -37, "B": -10, "C" :-15, "D":20, "E" :15}
Person_d = {"A" : -21, "B": 35, "C" :-14, "D":10, "E" :-7}
Person_e = {"A" : 29, "B": -23, "C" :-19, "D":25, "E" :-10}

person_dic = {"a" : Person_a, "b" : Person_b, "c" : Person_c, "d" : Person_d, "e" : Person_e}

Orientation_a = {"A" : 1, "B": -3, "C" : 5, "D":-3, "E" :-2}
Orientation_b  = {"A" : 1, "B": -2, "C" :3, "D":-1, "E" :1}
Orientation_c = {"A" : 2, "B": -3, "C" :3, "D":-3, "E" :-1}
Orientation_d  = {"A" : 2, "B": -5, "C" :4, "D":-1, "E" :0}
Orientation_e  = {"A" : 3, "B": -2, "C" :4, "D":-2, "E" :2}

orientation_dic = {"a" : Orientation_a, "b" : Orientation_b, "c" : Orientation_c, "d" :Orientation_d, "e" : Orientation_e}

Distance_dic = {"I":0.45,"Pe":1.2,"S":3.6,"Pu":7.6}

#figure setting 
#fig1 = plt.figure(1)
#ax = plt.axes(xlim =(0,100),ylim = (0,1000))
#ax.set_aspect("equal")
#fig2 = plt.figure(2)
#ax = plt.axes(xlim =(0,30),ylim = (0,100))
# set equal aspect such that the circle is not shown as ellipse

x_vals = []
y_vals = []
energy_vals = []
energyX_vals = []

Random function generation and energy decrease function 


In [3]:
def ramdonGenerator():
    return random.randint(30,80)

In [4]:
def ramdonGenerator2():
    return random.randint(0,100)

In [5]:
def ramdonLocationGenerator():
    return  random.randint(0,200)

In [6]:
def energy_decay(personalType,step,person_speed_rate):
    
    decayRate = person_speed_rate[personalType]   
    energy_decay = inital_energy - inital_energy * decayRate * np.exp(-step/40)*0.5
    
    return energy_decay

Person agent class 

Person Class has simulation person informaction:

Energy, orientation, location,personal type

In [7]:
class Person:
    
    def __init__(self,intital_energy,action_table,start_location_Y,start_location_X,person_Type,inital_speed,OrientationTable):
        
        self.init_energy = intital_energy
        self.actions = action_table
        self.cur_speed = inital_speed;
        self.cur_energy = intital_energy
        self.location_Y = start_location_Y
        self.personType = person_Type
        self.actionTable = action_table
        
        self.location_X = start_location_X
        self.orientation = 90
        self.OrientationTable = OrientationTable
        self.walking_speed = 50
        
    def setSpeed(self):
        walking_speed = self.walking_speed
        if self.cur_energy < 10 or self.cur_energy > 90:
            self.cur_speed = 0*walking_speed
        elif self.cur_energy <= 60 and self.cur_energy >= 40:
            self.cur_speed = -1/16*(40/10-1)*(40/10-9)*walking_speed
        else:
             self.cur_speed = -1/16*(self.cur_energy/10-1)*(self.cur_energy/10-9)*walking_speed
                
        
        if self.cur_speed <= 0:
            self.cur_speed = 0
        if self.cur_speed >= walking_speed:
            self.cur_speed = walking_speed
     
            
        
    def setEnergy(self,current_energy):
        self.cur_energy = current_energy
        if self.cur_energy > 100:
            self.cur_energy = 100
            
        elif self.cur_energy < 0:
            self.cur_energy = 0
        
        
        
     
    def setLocation_Y(self):
        
        self.location_Y = self.location_Y + self.cur_speed*person_moving_rate[self.personType]*math.sin(self.orientation*math.pi/180)
        
        if self.location_Y >= Goal:
            self.location_Y = Goal
        elif self.location_Y <= 0:
             self.location_Y = 0
        
        
    def setLocation_X(self):
        self.location_X = self.location_X + self.cur_speed*person_moving_rate[self.personType]*math.cos(self.orientation*math.pi/180)
        
        if self.location_X > 1000:
            self.location_X = 1000
        elif self.location_X < 0:
            self.location_X = 0
            
    def setOrientation(self,orientation_change):
    
        self.orientation = self.orientation + orientation_change
        
        if self.orientation > 180:
            self.orientation = -(360-self.orientation)
        if self.orientation > 360:
            self.orientation = self.orientation - 360
        if self.orientation < -180:
            self.orientation = 360 + self.orientation
        if self.orientation < -360:
            self.orientation = self.orientation + 360
        
        
        
        
        
        
    
    def getSpeed(self):
        return self.cur_speed 
    
    def getEnergy(self):
        return self.cur_energy
    
    def getLocation_Y(self):
        return self.location_Y
    def getLocation_X(self):
        return self.location_X

    def getOrientation(self):
        return self.orientation
    
    
    
    def getPersonType(self):
        return self.personType
    
    def getPersonNumber(self):
        
        list_t = {"a" : 0, "b" : 1, "c" : 2, "d" : 3, "e" : 4}
        person_number = list_t[self.personType] 
        return person_number
    
    def getActionTable(self):
        return self.actionTable
    
    def getOrientationTable(self):
        return self.OrientationTable
    
    
    
    
    


Robot agent class 

The robot agent only contain 2 information 

Location and the Soical distance

In [8]:
class Robort:
    
    def __init__(self,start_location_Y,start_location_X,distance_human):
        
        self.location_Y = start_location_Y
        self.location_X = start_location_X
        self.distance_human = distance_human
        
    def setLocation_Y(self,locationY):
        self.location_Y = locationY
        
  
    def setLocation_X(self,locationX):
        self.location_X = locationX
        
    
    def setdistance_human(self,distance_human):
        self.distance_human = distance_human
        
    def getLocation_Y(self):
        return self.location_Y
    
    def getLocation_X(self):
        return self.location_X
    
    def getdistance_human(self):
        return self.distance_human
    

Here to generator a person agent to start the training and simulation 

In [9]:
# inintal all the varible randomly

inital_energy = ramdonGenerator()
inital_speed =  ramdonGenerator()
start_location_Y = ramdonLocationGenerator()
start_location_X = 500
start_distance_human = 5.0

# enter person type: "a,b,c,d,e"
PrsonType = input()
Person_type = person_dic[PrsonType]
orientationTable = orientation_dic[PrsonType]
sim_Person = Person(inital_energy,Person_type,start_location_Y,start_location_X,PrsonType,inital_speed,orientationTable)
sim_Robot = Robort(sim_Person.getLocation_Y(),sim_Person.getLocation_X(),start_distance_human)

a


RL network for Soical norms traninig 

In [10]:
 class SoicalNorm(gym.Env):
    def __init__(self,env_config):
        #Action we can take, A,B,C,D,E
        self.action_space = Discrete(4)
        #Energy array 
        self.observation_space = Box(low=np.array([0.0,0.0]),high = np.array([10.0,100.0]),dtype = np.float32)
        #set start Enegry
        self.state = [sim_Robot.getdistance_human(),sim_Person.getEnergy()]
        self.Step = 0
        
        
    def step(self, action):
        
     
        
  
        
        self.Step += 1
        a_list = ["I","Pe","S","Pu"]
        
        # The action range
        # Example:
        #if the action is 2:
        #The robot will pick the range in 1.2m to 3.6m
        
        if action == 0:
            range_pick_upper = Distance_dic[a_list[action]]
            range_pick_lower  = 0
        else:
            range_pick_upper = Distance_dic[a_list[action]]
            range_pick_lower = Distance_dic[a_list[action-1]]
        
        
        #Reward 
        
        reward = 0
        if sim_Person.getEnergy() >= 40 and sim_Person.getEnergy() <= 60:
            if action == 0:
                reward = -100
            elif action == 1:
                reward = 100
            elif action == 2:
                reward = 10
            elif action == 3:
                reward = -10
                
        if sim_Person.getEnergy() < 40 and sim_Person.getEnergy() >= 10:
            if action == 0:
                reward = -100
            elif action == 1:
                reward = 10
            elif action == 2:
                reward = 100
            elif action == 3:
                reward = -10
                
        if sim_Person.getEnergy() > 60 and sim_Person.getEnergy() <= 90:
            if action == 0:
                reward = -100
            elif action == 1:
                reward = -10
            elif action == 2:
                reward = 10
            elif action == 3:
                reward = 100
                
        if sim_Person.getEnergy() < 10 and sim_Person.getEnergy() > 90:
            if action == 0:
                reward = -100
            elif action == 1:
                reward = -100
            elif action == 2:
                reward = -100
            elif action == 3:
                reward = 100
        
        # For the action range, generate a random number
        
        human_distance =  random.random() * (range_pick_upper -  range_pick_lower) +  range_pick_lower
        sim_Robot.setdistance_human(human_distance)
        
        # End condition for one eposide
        if self.Step <= 200: 
            done = False
        else:
            done = True
        
            
        #if you are traning the network uncommond below code, if for test commond the below code:
        
        #sim_Person.setEnergy(ramdonGenerator2())
        
        
        self.state = [sim_Robot.getdistance_human(),sim_Person.getEnergy()]

        info = {}
        
        return self.state,reward,done,info  
        
    def reset(self):
        inital_energy = ramdonGenerator()
        inital_speed = inital_energy
        start_location_Y = ramdonLocationGenerator()
        start_location_X = 500
        typePerson = random.randint(0,4)
        personList = ["a","b","c","d","e"]
        PrsonType = personList[typePerson]
        Person_type = person_dic[PrsonType]
        orientationTable = orientation_dic[PrsonType]
        start_distance_human = 3.6
        sim_Person = Person(inital_energy,Person_type,start_location_Y,start_location_X,PrsonType,inital_speed,orientationTable)
        sim_Robot = Robort(sim_Person.getLocation_Y(),sim_Person.getLocation_X(),start_distance_human)
        
        self.state = [sim_Robot.getdistance_human(),sim_Person.getEnergy()]
        
        return self.state
    
    def render():
        return
    
    def setTeststate(self,energy):
        self.state = [3.6,energy]
        
    

Here are the agent class tester to see if the agent is working or not


In [11]:
a_list = ["I","Pe","S","Pu"]
action = 1
if action == 0:
            range_pick_upper = Distance_dic[a_list[action]]
            range_pick_lower  = 0
else:
            range_pick_upper = Distance_dic[a_list[action]]
            range_pick_lower = Distance_dic[a_list[action-1]]
            
human_distance = random.random() * (range_pick_upper -  range_pick_lower) +  range_pick_lower;
print(range_pick_upper)
sim_Robot.setdistance_human(human_distance)
print(sim_Robot.getdistance_human())

print(human_distance)
sim_Person.setEnergy(ramdonGenerator())
print(sim_Person.getEnergy())

1.2
0.5602829252698659
0.5602829252698659
52


Here start the set up for SoicalNorm RL traning

In [36]:
register_env("SoicalNorm", lambda config:SoicalNorm(config))

In [ ]:
trainer =ppo.PPOTrainer(env = "SoicalNorm")

N_ITER = 10
pp = pprint.PrettyPrinter()

for n in range(N_ITER):
    result = trainer.train()
    #pprint.pprint(result)
    file_name = trainer.save()
    print(file_name)

Reload the trained network 

for checkpoint_path plz replace the path where your trained ntwork saved

In [37]:
agent_social = ppo.PPOTrainer(env = "SoicalNorm")
checkpoint_path = r"C:\Users\jdu39/ray_results\PPO_SoicalNorm_2021-10-14_20-35-52w1qb82bu\checkpoint_000010\checkpoint-10"
agent_social.restore(checkpoint_path)

(pid=27632) C:\Users\jdu39\anaconda3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=27632)   warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
(pid=45764) C:\Users\jdu39\anaconda3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=45764)   warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
2021-10-22 19:56:20,688	INFO trainable.py:106 -- Trainable.setup took 21.520 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=33868) Windows fatal exception: access violation
(pid=33868) 
(pid=10124) Windows fatal exception: access violation
(pid=10124) 
2021-10-22 19:56:20,801	INFO trainable.py:382 -- Restored on 192.168.0.115 from checkpoint: C:\Users\jdu39/ray_results\PPO_SoicalNorm_2021-10-14_20-35-52w1qb82bu\checkpoint_000010\checkpoint-10
2021-10-22 19:56:20,802	INFO


The main RL network 

It is use to train and test RL agent to pick a action to keep energy and orientation in a desired range. 

The action space is Discrete (0,1,2,3,4,5)

The observation space is continuous box space for 4 states: energy, orientation,speed and personal type






In [12]:
class MyEnv(gym.Env):
    def __init__(self,env_config):
        #Action we can take, A,B,C,D,E
        self.action_space = Discrete(5)
        #Energy array 
        self.observation_space = Box(low= np.array([0,0,-180,0]),high = np.array([100,100,180,4]),dtype = np.float64)
        #set start Enegry
        self.state = [sim_Person.getEnergy(),sim_Person.getSpeed(),sim_Person.getOrientation(),sim_Person.getPersonNumber()]
        
        self.old_state = self.state
        #set goal
        
        self.goal = Goal
        self.goaldistance = Goal
        self.Step = 0
        
        self.person = sim_Person
        
    
        
    def step(self, action):
        #step in each iteration 
        
        self.Step += 1
        
        #apply action 
        a_list = ["A","B","C","D","E"]
        
        # energy decay each step
        #energy_change = self.state[0]-self.state[1]/10
        #self.person.setEnergy(energy_change)
        self.state[0] = self.person.getEnergy()
        
        
        
        #energy change after action
        
        Person_type = self.person.getActionTable()
        orientation_type = self.person.getOrientationTable()
        
        
        orientation_change = orientation_type[a_list[action]]
        energy_change = self.state[0]+Person_type[a_list[action]]
        self.person.setEnergy(energy_change)
        self.state[0] = self.person.getEnergy()
        
    

        
        #update the person class 
        self.person.setSpeed()
        self.person.setOrientation(orientation_change)
        self.person.setLocation_Y()
        self.person.setLocation_X()
        
        
         # how far to the goal
        self.goaldistance = self.goal - self.person.getLocation_Y()
        
        #set point in figure 
        # create a point in the axes

        self.state[1] = self.person.getSpeed()
        self.state[2] = self.person.getOrientation()
        self.state[3] = self.person.getPersonNumber()
        
      
        
        #calculating reward
        # reward function is desirbe in the final report. 
        
        reward = 0
        speed_reward = 0
        
        max_reward = 35
        min_reward = -50
        scale_factor = 10
        
        if (self.state[0] <= 60 and self.state[0] >= 40) and (self.state[2] <= 95 and self.state[2] >= 85):
            reward = max_reward 
            speed_reward = self.state[1]/46.875 * max_reward
        elif (self.state[0] < 10 or self.state[0] > 90) and (self.state[2] < 45  or self.state[2] > 135):
            reward = min_reward
            speed_reward = 0
            
        elif (self.state[0] < 10 or self.state[0] > 90) or (self.state[2] < 45  or self.state[2] > 135):
            if (self.state[0] < 10 or self.state[0] > 90) and (self.state[2] > 45 and self.state[2] < 135):
                reward = min_reward*(1+1/20.25*(self.state[2]/scale_factor-4.5)*(self.state[2]/scale_factor-13.5))
                speed_reward = 0
            elif (self.state[2] < 45 or self.state[2] > 135) and (self.state[0] > 10 and self.state[0] < 90):
                reward = min_reward*(1+1/16*(self.state[0]/scale_factor-1)*(self.state[0]/scale_factor-9))
                speed_reward = self.state[1]/46.875* max_reward
        else:
            reward = -(self.state[0]/scale_factor-1)*(self.state[0]/scale_factor-9) - (self.state[2]/scale_factor-4.5)*(self.state[2]/scale_factor-13.5)
            speed_reward = self.state[1]/46.875* max_reward
            
        reward += speed_reward 

        
        #check if the walking done
        Timeout = False
        done = False
        if  self.goaldistance <= 0:
            done = True
        elif self.Step > 50:
            done = True
            Timeout = True
        
                
        info = {}
        
 
        
        
        
        self.old_state = self.state
        
        
        
        
        
        return self.state,reward,done,info   
        
    def reset(self):
        
        #reset all the parameter 
        inital_speed = ramdonGenerator()
        inital_energy = ramdonGenerator()
        start_location_Y = ramdonLocationGenerator()
        start_location_X = 500
        typePerson = random.randint(0,4)
        personList = ["a","b","c","d","e"]
        PrsonType = personList[typePerson]
        Person_type = person_dic[PrsonType]
        orientationTable = orientation_dic[PrsonType]
        
        sim_Person = Person(inital_energy,Person_type,start_location_Y,start_location_X,PrsonType,inital_speed,orientationTable)
        

        self.state = [sim_Person.getEnergy(),sim_Person.getSpeed(),sim_Person.getOrientation(),sim_Person.getPersonNumber()]
        self.goal = Goal
        self.goaldistance = Goal
        self.Step = 0
        
        self.person = sim_Person
        self.old_state = self.state
        
            
        return self.state
        
        
        
    def render(self):
        return 
        
    def getPerson(self):
        return self.person 

To inital ray library 

In [12]:
ray.init()

C:\Users\jdu39\anaconda3\lib\site-packages\ray\_private\services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


{'node_ip_address': '192.168.0.115',
 'raylet_ip_address': '192.168.0.115',
 'redis_address': '192.168.0.115:6379',
 'object_store_address': 'tcp://127.0.0.1:59891',
 'raylet_socket_name': 'tcp://127.0.0.1:52799',
 'webui_url': None,
 'session_dir': 'C:\\Users\\jdu39\\AppData\\Local\\Temp\\ray\\session_2021-10-22_19-30-46_152329_43568',
 'metrics_export_port': 56903,
 'node_id': 'a8bfbdb4bb3fca4256b803e3203b04a55346c0e13d81eb974ef6886d'}

To shutdown ray library 

In [ ]:
#uncommond it if you want to shutdown ray 

#ray.shutdown()

training Set up for main RL agent

In [39]:
register_env("MyEnv", lambda config:MyEnv(config))

The policy network sturcture can be found here 

In [27]:
policy = trainer.get_policy()
model = policy.model
print(model.base_model.summary())

NameError: name 'trainer' is not defined

Traning agent 

In [ ]:
trainer =ppo.PPOTrainer(env = "MyEnv")

index = count()

N_ITER = 30
pp = pprint.PrettyPrinter()

for n in range(N_ITER):
    result = trainer.train()
    #pprint.pprint(result)
    file_name = trainer.save()
    print(file_name)
    
    


Reload the trained network

for checkpoint_path plz replace the path where your trained network saved

In [40]:
agent = ppo.PPOTrainer(env = "MyEnv")
checkpoint_path = r"C:\Users\jdu39/ray_results\PPO_MyEnv_2021-10-15_14-34-30ozo1cjc6\checkpoint_000030\checkpoint-30"
agent.restore(checkpoint_path)

(pid=22108) C:\Users\jdu39\anaconda3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float64
(pid=22108)   warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
(pid=24060) C:\Users\jdu39\anaconda3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float64
(pid=24060)   warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
2021-10-22 19:57:04,345	INFO trainable.py:106 -- Trainable.setup took 19.899 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=33124) Windows fatal exception: access violation
(pid=33124) 
(pid=11104) Windows fatal exception: access violation
(pid=11104) 
2021-10-22 19:57:04,504	INFO trainable.py:382 -- Restored on 192.168.0.115 from checkpoint: C:\Users\jdu39/ray_results\PPO_MyEnv_2021-10-15_14-34-30ozo1cjc6\checkpoint_000030\checkpoint-30
2021-10-22 19:57:04,508	INFO trai

Here is for testing 

remember to commonet out the RandomGeneration() in the soical norm network 

In [41]:
#testing 

Goal = 500
Reward_epic = []
Energy_epic = []
Speed_epic = []
orientation_epic = []
number_epic = []
Location_X_epic = []
Location_Y_epic = []
episode_reward_epic = []
soical_reward_epic = []
human_distance_epic = []


env = MyEnv({})
social_env = SoicalNorm({})


# run until episode ends
episode_reward = 0
step = 0
done = False
obs = env.reset()
soical_obs = [3.6,obs[0]]
#print(obs)
#print(soical_obs)

for i in range(50):
    while not done:
        action = agent.compute_single_action(obs)
        action_soical = agent_social.compute_single_action(soical_obs)
        
        obs, reward, done, info = env.step(action)
        soical_obs, soical_reward,d,i = social_env.step(action_soical)
        
        person_test = env.getPerson()
        Location_X = person_test.getLocation_X()
        Location_X_epic.append(Location_X)
        
        Location_Y = person_test.getLocation_Y()
        Location_Y_epic.append(Location_Y)
        
        human_distance_epic.append(soical_obs[0])
        soical_reward_epic.append(soical_reward)
        
        Reward_epic.append(reward)
        Energy_epic.append(obs[0])
        Speed_epic.append(obs[1])
        orientation_epic.append(obs[2])
        
        episode_reward += reward
        
        step += 1
    
    number_epic.append(step)
    episode_reward_epic.append(episode_reward)
    
    episode_reward = 0 
    step = 0
    done = False    
    obs = env.reset()
    social_env.setTeststate(obs[0])
    soical_obs = [3.6,obs[0]]


if you want to see any data, plz uncomment it 

In [ ]:
#print(Reward_epic)
#print(Energy_epic)
#print(Speed_epic)
#print(Location_X_epic)
#print(Location_Y_epic)
#print(orientation_epic)
#print(number_epic)
#print(episode_reward_epic)
#print(human_distance_epic)



To find the percentage of values in a desire range

In [ ]:
COUNT = 0
for element in orientation_epic:
    if element >= 80 and element <= 100:
        COUNT += 1
    
correct = COUNT/len(orientation_epic)
print(correct)

In [ ]:
COUNT = 0
for element in number_epic:
        COUNT += element
    
correct = COUNT/len(number_epic)
print(correct)

Here is to launch Tensorflow

In [ ]:
%load_ext tensorboard
import tensorflow as tf
import datetime, os
from tensorboard import notebook
notebook.list() 
%tensorboard --logdir=tensorboard --logdir=~/ray_results

%tensorboard dev upload --logdir \ '~/ray_results'

Here is all the plots and animation 

In [ ]:

fig, (ax1, ax2,ax3,ax4) = plt.subplots(1,4)
ax1.title.set_text("energy reward function behaviour")
ax2.title.set_text("speed reward function behaviour")
ax3.title.set_text("energy vs speed behaviour")
ax4.title.set_text("orientation reward function behaviour")
ax1.plot(Energy_epic, Reward_epic,'bo')
ax2.plot(Speed_epic, Reward_epic,'bo')
ax3.plot(Energy_epic,Speed_epic,'bo')
ax4.plot(orientation_epic,Reward_epic,'bo')
ax1.set(xlabel='Energy', ylabel='Reward')
ax2.set(xlabel='Speed', ylabel='Reward')
ax3.set(xlabel='Energy', ylabel='Speed')
ax4.set(xlabel='orientation', ylabel='reward')




In [ ]:
plot1 = plt. figure(2)
plt.plot(range(len(Energy_epic)),Energy_epic)
plt.xlabel("number of steps")
plt.ylabel("energy")
plt.title("energy_trained")


plot2 = plt. figure(3)
plt.plot(range(len(orientation_epic)),orientation_epic)
plt.xlabel("number of steps")
plt.ylabel("orientation")
plt.title("orientation_trained")

plot3 = plt. figure(4)
plt.plot(range(len(Reward_epic)),Reward_epic,'bo')
plt.xlabel("number of steps")
plt.ylabel("reward_epic")
plt.title("Reward_trained")

plot4 = plt. figure(5)
plt.plot(range(len(number_epic)),number_epic,'bo')
plt.xlabel("number of steps")
plt.ylabel("the iterations each episode")
plt.title("number_epic_trained")

plot5 = plt. figure(6)
plt.plot(range(len(episode_reward_epic)),episode_reward_epic,'bo')
plt.xlabel("number of steps")
plt.ylabel("total reward per episode")
plt.title("episode_reward_trained")
plt.ylim(400,2700)

plot6 = plt. figure(7)
plt.plot(range(len(Speed_epic)),Speed_epic,'bo')
plt.xlabel("number of steps")
plt.ylabel("speed")
plt.title("speed_trained")

plot7 = plt. figure(8)
plt.hist(Speed_epic, bins=100)
plt.gca().set(title='Speed Histogram trained', ylabel='Frequency');

plot7 = plt. figure(9)
plt.hist(Energy_epic, bins=100)
plt.gca().set(title='Energy Histogram trained', ylabel='Frequency');

plot7 = plt. figure(10)
plt.hist(orientation_epic, bins=100)
plt.gca().set(title='orientation Histogram trained', ylabel='Frequency');

plot7 = plt. figure(11)
plt.hist(Reward_epic, bins=100)
plt.gca().set(title='reward Histogram trained', ylabel='Frequency');

Animation 

In [42]:
fig4, ax = plt.subplots()
# set the axes limits
ax.axis([0,1000,-5,500])
# set equal aspect such that the circle is not shown as ellipse
ax.set_aspect("equal")
# create a point in the axes
points = []
points.append(ax.plot(500,10, marker="o")[0])
points.append(ax.plot(500,10-3.6, marker="o")[0])

energy_text = ax.text(0.02, 0.95, '', transform = ax.transAxes)
distance_text = ax.text(0.02, 0.90, '', transform = ax.transAxes)
orientation_text = ax.text(0.02, 0.85, '', transform = ax.transAxes)
speed_text = ax.text(0.02, 0.80, '', transform=ax.transAxes)


def update_dot(i):
    x = Location_X_epic[i]
    y = Location_Y_epic[i]
    
    X =  Location_X_epic[i]
    Y = Location_Y_epic[i] - human_distance_epic[i]*10
    
    energy_text.set_text('energy = %.2f' % Energy_epic[i])
    distance_text.set_text('distance = %.2f' % human_distance_epic[i])
    speed_text.set_text('speed = %.2f' % Speed_epic[i])
    orientation_text.set_text('orientation = %.2f' % orientation_epic[i])
    
    points[0].set_data([x],[y])
    points[1].set_data([X],[Y])
    
    return points,energy_text,distance_text,speed_text,orientation_text

animition2 = animation.FuncAnimation(fig4,update_dot,interval = 500 ,blit = False)
plt.title("Simulation")
plt.legend(["Human","Robot"])

plt.show()

2021-10-22 20:03:14,777	ERROR worker.py:475 -- print_logs: Connection closed by server.
2021-10-22 20:03:14,778	ERROR worker.py:1217 -- listen_error_messages_raylet: Connection closed by server.
2021-10-22 20:03:14,778	ERROR import_thread.py:88 -- ImportThread: Connection closed by server.


Random agent simulation here

In [13]:
Goal = 500
Reward_epic_random = []
Energy_epic_random = []
Speed_epic_random = []
orientation_epic_random = []
number_epic_random = []
Location_X_epic_random = []
Location_Y_epic_random = []
episode_reward_epic_random = []
env = MyEnv({})

# run until episode ends
episode_reward = 0
step = 0
done = False
obs = env.reset()
print(obs)
for i in range(50):
    while not done:
        action = random.randint(0,4)
        obs, reward, done, info = env.step(action)
        person_test = env.getPerson()
        Location_X = person_test.getLocation_X()
        Location_X_epic_random.append(Location_X)
        
        Location_Y = person_test.getLocation_Y()
        Location_Y_epic_random.append(Location_Y)
        
        Reward_epic_random.append(reward)
        Energy_epic_random.append(obs[0])
        Speed_epic_random.append(obs[1])
        orientation_epic_random.append(obs[2])
        
        episode_reward += reward
        
        step += 1
    
    number_epic_random.append(step)
    episode_reward_epic_random.append(episode_reward)
    
    episode_reward = 0 
    step = 0
    done = False    
    obs = env.reset()


[32, 62, 90, 4]


C:\Users\jdu39\anaconda3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float64
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


Plot and animation and data analysis 

In [ ]:
COUNT = 0
for element in orientation_epic_random:
    if element >= 80 and element >= 100:
        COUNT += 1
    
correct = COUNT/len(orientation_epic_random)
print(correct)

In [ ]:
COUNT = 0
for element in number_epic_random:
        COUNT += element
    
correct = COUNT/len(number_epic_random)
print(correct)

In [ ]:
plot1 = plt. figure(2)
plt.plot(range(len(Energy_epic_random)),Energy_epic_random)
plt.xlabel("number of steps")
plt.ylabel("energy")
plt.title("energy_random")


plot2 = plt. figure(3)
plt.plot(range(len(orientation_epic_random)),orientation_epic_random)
plt.xlabel("number of steps")
plt.ylabel("orientation")
plt.title("orientation_random")


plot3 = plt. figure(4)
plt.plot(range(len(Reward_epic_random)),Reward_epic_random,'bo')
plt.xlabel("number of steps")
plt.ylabel("reward_epic")
plt.title("Reward_random")


plot4 = plt. figure(5)
plt.plot(range(len(number_epic_random)),number_epic_random,'bo')
plt.xlabel("number of steps")
plt.ylabel("the iterations each episode")
plt.title("number_epic_random")


plot5 = plt. figure(6)
plt.plot(range(len(episode_reward_epic_random)),episode_reward_epic_random,'bo')
plt.xlabel("number of steps")
plt.ylabel("total reward per episode")
plt.title("episode_reward_random")
plt.ylim(400,2700)

plot1 = plt. figure(7)
plt.plot(range(len(Speed_epic_random)),Speed_epic_random,'bo')
plt.xlabel("number of steps")
plt.ylabel("speed")
plt.title("speed_random")

plot8 = plt. figure(8)
plt.hist(Speed_epic_random, bins=100)
plt.gca().set(title='Speed Histogram random', ylabel='Frequency');

plot7 = plt. figure(9)
plt.hist(Energy_epic_random, bins=100)
plt.gca().set(title='Energy Histogram random', ylabel='Frequency');

plot7 = plt. figure(10)
plt.hist(orientation_epic_random, bins=100)
plt.gca().set(title='orientation Histogram random', ylabel='Frequency');

plot7 = plt. figure(11)
plt.hist(Reward_epic_random, bins=100)
plt.gca().set(title='reward Histogram random', ylabel='Frequency');


In [17]:
fig3, ax = plt.subplots()
# set the axes limits
ax.axis([0,1000,0,1000])
# set equal aspect such that the circle is not shown as ellipse
ax.set_aspect("equal")
# create a point in the axes
points = []
points.append(ax.plot(500,10, marker="o")[0])
points.append(ax.plot(500,10-3.6, marker="o")[0])

energy_text = ax.text(0.02, 0.95, '', transform = ax.transAxes)
distance_text = ax.text(0.02, 0.90, '', transform = ax.transAxes)
orientation_text = ax.text(0.02, 0.85, '', transform = ax.transAxes)
speed_text = ax.text(0.02, 0.80, '', transform=ax.transAxes)

def update_dot(i):
    x = Location_X_epic_random[i]
    y = Location_Y_epic_random[i]
    
    X =  Location_X_epic_random[i]
    Y = Location_Y_epic_random[i] - 2.5*10
    
    energy_text.set_text('energy = %.2f' % Energy_epic_random[i])
    speed_text.set_text('speed = %.2f' % Speed_epic_random[i])
    orientation_text.set_text('orientation = %.2f' % orientation_epic_random[i])
    
    points[0].set_data([x],[y])
    points[1].set_data([X],[Y])
    
    return points,energy_text,speed_text,orientation_text

plt.title("Simulation")
plt.legend(["Human","Robot"])
animition1 = animation.FuncAnimation(fig3,update_dot,interval = 500 ,blit = False)
    
plt.show()